In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hazetestvideo/hazevideo1.mp4
/kaggle/input/testvideo2/video.mp4
/kaggle/input/synthetic-objective-testing-set-sots-reside/metadata_indoor.csv
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1413.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1445.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1401.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1420.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1405.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1430.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1435.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1447.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1423.png
/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear/1402.png
/kaggle/input/synthetic-objective-testing-se

In [30]:
import cv2
import numpy as np
from os.path import isfile

def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1
    cap.release()
    return frame_array

In [31]:
def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

def dehaze_images(frame_array):
    dehazed_frames = []
    for frame in frame_array:
        dehazed_frame = atmosdehaze(frame)
        dehazed_frames.append(dehazed_frame)

    # Convert the dehazed frames to an array
    dehazed_array = np.array(dehazed_frames)
    return dehazed_array

In [32]:
import cv2
import os
import shutil
from ultralytics import YOLO
def objectdectect(source_images):
    output_folder = 'temp_images'
    os.makedirs(output_folder, exist_ok=True)

    # Save the images from the array to the folder
    for i, image in enumerate(source_images):
        image_filename = os.path.join(output_folder, f'image_{i}.jpg')
        cv2.imwrite(image_filename, image)

    model = YOLO('yolov8n.pt')  # Pretrained YOLOv8n model
    model(source='temp_images', save=True, project='Project')
    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    # Create an empty list to store the new images with bounding boxes
    output_images = []

    # Define the directory where the saved results are located
    project_directory = 'project'

    # List the files in the 'project/predict' directory
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    # Sort the image files by name
    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))


    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in sorted_result_files]
    shutil.rmtree('/kaggle/working/Project')
    return output_images

In [33]:
def dehazed2video(dehazed_array, pathOut, fps=30):
    size = (dehazed_array[0].shape[1], dehazed_array[0].shape[0])
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(dehazed_array)):
        out.write(dehazed_array[i])
    out.release()

In [35]:
def main(input_video, output_video):
    frame_array = video2framesarray(input_video)
    dehazed_array = dehaze_images(frame_array)
    n = input("Do you want Object Detection: (y/n): ")
    if (n == 'y'):
        dehazed_array = objectdectect(dehazed_array)
        dehazed2video(dehazed_array, output_video)
    else:
        dehazed2video(dehazed_array, output_video)

if __name__ == '__main__':
    input_video = '/kaggle/input/hazetestvideo/hazevideo1.mp4'
    output_video = 'output_video.mp4' 
    main(input_video, output_video)

Do you want Object Detection: (y/n):  y



image 1/451 /kaggle/working/temp_images/image_0.jpg: 640x384 1 boat, 89.7ms
image 2/451 /kaggle/working/temp_images/image_1.jpg: 640x384 1 boat, 82.4ms
image 3/451 /kaggle/working/temp_images/image_10.jpg: 640x384 (no detections), 82.4ms
image 4/451 /kaggle/working/temp_images/image_100.jpg: 640x384 (no detections), 82.1ms
image 5/451 /kaggle/working/temp_images/image_101.jpg: 640x384 (no detections), 83.2ms
image 6/451 /kaggle/working/temp_images/image_102.jpg: 640x384 (no detections), 87.3ms
image 7/451 /kaggle/working/temp_images/image_103.jpg: 640x384 (no detections), 84.0ms
image 8/451 /kaggle/working/temp_images/image_104.jpg: 640x384 (no detections), 81.7ms
image 9/451 /kaggle/working/temp_images/image_105.jpg: 640x384 (no detections), 83.4ms
image 10/451 /kaggle/working/temp_images/image_106.jpg: 640x384 (no detections), 85.5ms
image 11/451 /kaggle/working/temp_images/image_107.jpg: 640x384 (no detections), 84.0ms
image 12/451 /kaggle/working/temp_images/image_108.jpg: 640x384

In [7]:
def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1
    cap.release()
    return frame_array

In [12]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.202 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5016.7/8062.4 GB disk)


In [27]:
import cv2
import os
import shutil
from ultralytics import YOLO
def objectdectect(source_images):
    output_folder = 'temp_images'
    os.makedirs(output_folder, exist_ok=True)

    # Save the images from the array to the folder
    for i, image in enumerate(source_images):
        image_filename = os.path.join(output_folder, f'image_{i}.jpg')
        cv2.imwrite(image_filename, image)

    model = YOLO('yolov8n.pt')  # Pretrained YOLOv8n model
    model(source='temp_images', save=True, project='Project')
    #!yolo task=detect mode=predict model=yolov8n.pt source="temp_images"
    # Create an empty list to store the new images with bounding boxes
    output_images = []

    # Define the directory where the saved results are located
    project_directory = 'project'

    # List the files in the 'project/predict' directory
    result_files = os.listdir(os.path.join('/kaggle/working/Project/predict'))

    # Sort the image files by name
    sorted_result_files = sorted(result_files, key=lambda x: int(os.path.splitext(x)[0].split('_')[1]))


    # Load the sorted image files into an array
    output_images = [cv2.imread(os.path.join('/kaggle/working/Project/predict', filename)) for filename in sorted_result_files]
    shutil.rmtree('/kaggle/working/Project')
    return output_images

In [28]:
vi = '/kaggle/input/hazetestvideo/hazevideo1.mp4'
source_images = video2framesarray(vi)
print(objectdectect(source_images))


image 1/207 /kaggle/working/temp_images/image_0.jpg: 640x384 1 boat, 96.8ms
image 2/207 /kaggle/working/temp_images/image_1.jpg: 640x384 1 boat, 88.2ms
image 3/207 /kaggle/working/temp_images/image_10.jpg: 640x384 1 boat, 83.5ms
image 4/207 /kaggle/working/temp_images/image_100.jpg: 640x384 1 truck, 93.0ms
image 5/207 /kaggle/working/temp_images/image_101.jpg: 640x384 1 truck, 107.0ms
image 6/207 /kaggle/working/temp_images/image_102.jpg: 640x384 (no detections), 83.6ms
image 7/207 /kaggle/working/temp_images/image_103.jpg: 640x384 (no detections), 81.9ms
image 8/207 /kaggle/working/temp_images/image_104.jpg: 640x384 (no detections), 81.5ms
image 9/207 /kaggle/working/temp_images/image_105.jpg: 640x384 (no detections), 81.3ms
image 10/207 /kaggle/working/temp_images/image_106.jpg: 640x384 (no detections), 82.4ms
image 11/207 /kaggle/working/temp_images/image_107.jpg: 640x384 1 truck, 89.5ms
image 12/207 /kaggle/working/temp_images/image_108.jpg: 640x384 1 truck, 84.6ms
image 13/207 /k

[array([[[ 43,  89, 113],
        [ 42,  88, 112],
        [ 42,  88, 112],
        ...,
        [ 61, 128, 161],
        [ 54, 123, 156],
        [ 54, 123, 156]],

       [[ 42,  88, 112],
        [ 42,  88, 112],
        [ 41,  87, 111],
        ...,
        [ 60, 127, 160],
        [ 54, 123, 156],
        [ 54, 123, 156]],

       [[ 42,  88, 112],
        [ 41,  87, 111],
        [ 41,  87, 111],
        ...,
        [ 61, 128, 161],
        [ 55, 124, 157],
        [ 54, 123, 156]],

       ...,

       [[ 61, 136, 174],
        [ 61, 136, 174],
        [ 61, 136, 174],
        ...,
        [ 37,  75,  99],
        [ 37,  75,  99],
        [ 37,  75,  99]],

       [[ 61, 136, 174],
        [ 61, 136, 174],
        [ 61, 136, 174],
        ...,
        [ 37,  75,  99],
        [ 37,  75,  99],
        [ 37,  75,  99]],

       [[ 61, 136, 174],
        [ 61, 136, 174],
        [ 61, 136, 174],
        ...,
        [ 37,  75,  99],
        [ 37,  75,  99],
        [ 37,  75,  99]

In [23]:
import shutil
shutil.rmtree('/kaggle/working/Project')

In [ ]:
from flask import Flask, request, jsonify
import cv2
import numpy as np

app = Flask(__name__)

def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

# Define the API endpoint for dehazing a video
@app.route('/dehaze', methods=['POST'])
def dehaze_video():
    # Check if the request contains a file
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'})

    file = request.files['file']

    # Check if the file is a video
    if file.filename == '':
        return jsonify({'error': 'No file selected'})

    if file and allowed_file(file.filename):
        # Save the video file
        video_path = 'input_video.mp4'
        file.save(video_path)

        # Process the video
        frame_array = video2framesarray(video_path)
        dehazed_array = dehaze_images(frame_array)

        # Save the dehazed video
        output_video_path = 'output_video.avi'
        dehazed2video(dehazed_array, output_video_path)

        return jsonify({'message': 'Video dehazed successfully'})

    return jsonify({'error': 'Invalid file format'})

if __name__ == '__main__':
    app.run()


In [ ]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
import shutil
import cv2
import numpy as np
import os
import asyncio

app = FastAPI()

def video2framesarray(videoinput):
    cap = cv2.VideoCapture(videoinput)
    frame_number = 0
    frame_array = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_array.append(frame)
        frame_number += 1
    cap.release()
    return frame_array

def atmosdehaze(frame):
    # Convert the image to floating point representation
    hazy_image = frame.astype(np.float32) / 255.0

    # Estimate the atmospheric light
    dark_channel = np.min(hazy_image, axis=2)
    atmospheric_light = np.percentile(dark_channel, 99)

    # Estimate the transmission map
    transmission = 1 - 0.95 * dark_channel / (atmospheric_light + 1e-6)  # Add a small epsilon value

    # Clamp the transmission values to [0, 1]
    transmission = np.clip(transmission, 0, 1)

    # Estimate the scene radiance
    scene_radiance = np.zeros_like(hazy_image)
    for channel in range(3):
        scene_radiance[:, :, channel] = (hazy_image[:, :, channel] - atmospheric_light) / (transmission + 1e-6) + atmospheric_light  # Add a small epsilon value

    # Clamp the scene radiance values to [0, 1]
    scene_radiance = np.clip(scene_radiance, 0, 1)

    # Convert the scene radiance back to 8-bit representation
    scene_radiance = (scene_radiance * 255).astype(np.uint8)
    return scene_radiance

def dehaze_images(frame_array):
    dehazed_frames = []
    for frame in frame_array:
        dehazed_frame = atmosdehaze(frame)
        dehazed_frames.append(dehazed_frame)

    # Convert the dehazed frames to an array
    dehazed_array = np.array(dehazed_frames)
    return dehazed_array

def dehazed2video(dehazed_array, pathOut, fps=30):
    size = (dehazed_array[0].shape[1], dehazed_array[0].shape[0])
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(dehazed_array)):
        out.write(dehazed_array[i])
    out.release()

def dehaze_video(input_path, output_path):
    frame_array = video2framesarray(input_path)
    dehazed_array = dehaze_images(frame_array)
    dehazed2video(dehazed_array, output_path)

@app.post("/dehaze")
async def dehaze_endpoint(file: UploadFile = File(...), fps: int = Form(...)):
    if file.content_type != "video/mp4":
        return {"error": "Only MP4 videos are supported."}

    input_video = "temp_video.mp4"
    output_video = "output_video.mp4"  # You can change the format as needed

    with open(input_video, "wb") as f:
        shutil.copyfileobj(file.file, f)

    await asyncio.to_thread(dehaze_video, input_video, output_video)

    return FileResponse(output_video, media_type="video/mp4")

if __name__ == '__main__':
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [33]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO('yolov8n.pt')  # Pretrained YOLOv8n model
!yolo task = detect mode=predict model=yolov8n.pt source="/kaggle/input/synthetic-objective-testing-set-sots-reside/indoor/clear"


Ultralytics YOLOv8.0.202 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Traceback (most recent call last):
  File "/opt/conda/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/cfg/__init__.py", line 444, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/model.py", line 242, in predict
    return self.predictor.predict_cli(source=source) if is_cli else self.predictor(source=source, stream=stream)
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/predictor.py", line 205, in predict_cli
    for _ in gen:  # running CLI inference without accumulating any outputs (do not modify)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 35, in generator_context
    response = gen.send(None)
  File "

In [3]:
import cv2
import numpy as np

def preprocess_image(image, model_input_shape):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (model_input_shape[2], model_input_shape[1]))  # Resize to match the model's input shape
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Define the post-processing function for dehazing
def postprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = (image * 255).astype(np.uint8)
    return image